In [1]:
import csv
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation, Flatten, SpatialDropout2D, ELU
from keras.layers import Convolution2D,MaxPooling2D,Cropping2D
from keras.layers.core import Lambda
from keras.optimizers import SGD,Adam,RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle


Using TensorFlow backend.
/Users/likangning/anaconda/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
###load data
##read from csv and store the lines from driveing_log.csv file
##then for each line, extract the path to the camera image
lines= []
def read_lines(driving_log_path, extract_lines):
    with open(driving_log_path) as csvfile:
        reader=csv.reader(csvfile)
        for line in reader:
            extract_lines.append(line)
    return extract_lines
lines =read_lines("/Users/likangning/Desktop/data/driving_log.csv",lines)
#print("lines",len(lines))
lines=lines[1:]
#print(lines)

In [3]:
train_samples,validation_samples =train_test_split(lines,test_size=0.05)

In [4]:
x=train_samples[0]

In [5]:
print(x[0].split("/")[-1])

center_2016_12_01_13_35_20_965.jpg


In [6]:
print(len(train_samples),len(validation_samples),len(lines))

7634 402 8036


In [7]:
###Preprocessing data...
###original input shape is (160,320,3)
###1.resize.. width=200
###2.crop..   heigth=66
###3.Normalized

In [8]:
new_imgSize=(66,200,3)
def resize1(z):
    height=160
    width=320
    
    factor=float(new_imgSize[1]/float(width))
    
    resized_size=(int(width*factor),int(height*factor))
    
    z=cv2.resize(z,resized_size)
    
    crop_height= resized_size[1]-new_imgSize[0]
    
    return z[crop_height:,:,:]

def rgb_to_yuv(x):    ####ACCORDING TO NVIDIA 
    return cv2.cvtColor(x,cv2.COLOR_RGB2YUV)
    
def normalize(image):
    ####Normalize the input between [-0.5,0.5]
    return image /255. -0.5

In [9]:
angle_correction=0.2

In [11]:
def generator(lines,batch_size=32):
    num_lines=len(lines)
    while 1: ## Loop forever so the generator never terminates
        shuffle(lines)
        for offset in range(0,num_lines,batch_size):
            batch_lines= lines[offset:offset+batch_size]
            images=[]
            angles=[]
            for batch_line in batch_lines:
                #name="/Users/likangning/Desktop/data/IMG/"+batch_line[0].split("/")[-1]
                #name="/Users/likangning/Desktop/raw_data/"+batch_line[0]
                #name=batch_line[0]
                #center_image =cv2.imread(name)
                #center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                center_angle= float(batch_line[3])
                left_angle= center_angle + angle_correction
                right_angle= center_angle - angle_correction
                
                for i in range(3):
                    source_path=batch_line[i]
                    file_name= source_path.split("/")[-1]
                    current_path="/Users/likangning/Desktop/data/IMG/" +file_name
                    image_bgr= cv2.imread(current_path)
                    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
                    images.append(image)
                    images.append(cv2.flip(image,1))
                  
                #images.append(center_image)
                angles.extend([center_angle,center_angle*-1,
                               left_angle,left_angle*-1,
                               right_angle,right_angle*-1])
              
        ###trim image to only see section with road
        ###convert the image to np.array
                x_train= np.array(images)
                y_train= np.array(angles)
        ###yield in generator means return in normal function
                yield shuffle(x_train,y_train)

        
### Compile and train the model using the generator function


In [ ]:
def generator(samples,sample_size,mode):
    num_samples=len(samples)
    if mode== "train":
        cameras=["center","left","right"]
    else:
        cameras=["center"]
        
    ##generator loop
    
    while 1:
        sklearnrnrnearnearnearn.utils.shuffle(samples,random_state=43)
        for offnum_samples_samples in range(0, num_samples,sample_size):
            batch_sample=samples[offset:offset+sample_size]
            
            images=[]
            angles=[]
            
            for sample in batch_samples:
                for cam in cameras:
                    if mode=="train":
                        augmentation=np.random.choice(["flipping","brightness","shift","none"])
                    else:
                        augmentation="none"
                    
                    image= None
                    angle= float(sample[3])
                    
                    if cam=="center":
                        image=cv2.imread("/Users/likangning/Desktop/data/"+sample[0])
                    elif cam=="left":
                        image=cv2.imread("/Users/likangning/Desktop/data/"+sample[1])
                        angle+=0.2
                    elif cam=="right":
                        image=cv2.imread("/Users/likangning/Desktop/data/"+sample[2])
                        angle-=0.2
                        
                        
                    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                    if augmentation=="flipping":
                        image=cv2.flip(image,1)
                        angle*=-1.0
                    elif augmentation=="brightness":
                        image= cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
                        image=np.array(image,dtype=np.float64)
                        brightness= np.random.uniform()+0.5
                        image[:,:,2]=image[:,:,2]*brightness
                        image[:,:,2][image[:,:,2]>255]=255
                        image=np.array(image,dtype=np.float64)
                        image= cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
                        
                    elif augmentation=="shift":
                        tran_x=np.random.randint(0,100) -50
                        angle += tran_x*0.004
                        trans_y=np.random.randint(0,40)-20
                        
                        trans_matrix=np.float32([[1,0,trans_x],[0,1,trans_y]])
                        image=cv2.warpAffine(image,trans_matrix,(320,160))
                        
                    image= image[50:140,0:320]
                    
                    image=cv2.resize(image,(200,66),interpolation=cv2.INTER_AREA)
                    images.append(np.reshape(image,(1, 66,200,3)))
                    angles.append(np.array([[angle]]))
                    
                        
            X_train=np.vstack(images)
            y_train=np.vstack(angles)
            yield sklearn.utils.shuffle(X_train,y_train,random_state=21)
                        

In [12]:
train_generator= generator(train_samples,batch_size=32)
validation_generator= generator(validation_samples,batch_size=32)

In [13]:
print(train_generator)

<generator object generator at 0x124381d00>


In [14]:
def resize(image):
    import tensorflow as tf
    return tf.image.resize_images(image,(66,200))

In [15]:
    model1= Sequential()
    
    model1.add(Cropping2D(cropping=((70,25),(0,0)),input_shape=(160,320,3)))
    
    model1.add(Lambda(resize))
    model1.add(Lambda(normalize))
    model1.add(Convolution2D(16,8,8,border_mode="same",subsample=(4,4)))
    #model1.add(MaxPooling2D(2,2))
    model1.add(Activation("relu"))
    
    model1.add(Convolution2D(32,5,5,border_mode="same",subsample=(2,2)))
    #model1.add(MaxPooling2D(2,2))
    model1.add(Activation("relu"))
    
    model1.add(Convolution2D(64,5,5,border_mode="same",subsample=(2,2)))
    #model1.add(MaxPooling2D(2,2))
    model1.add(Activation("relu"))
    
    
    model1.add(Convolution2D(64,3,3,border_mode="same",subsample=(1,1)))
    #model1.add(MaxPooling2D(2,2))
    model1.add(Activation("relu"))
    
    
    #model1.add(Convolution2D(64,3,3,border_mode="valid",subsample=(1,1)))
    #model1.add(MaxPooling2D(2,2))
    #model1.add(Activation("relu"))
    
    model1.add(Flatten())
    model1.add(Dropout(0.2))
    model1.add(Activation("relu"))
    
    
    #model1.add(Dense(200))
    #model1.add(Dropout(0.2))
    #model1.add(Activation("relu"))
    
    model1.add(Dense(512))
    model1.add(Dropout(0.5))
    model1.add(Activation("relu"))
    
    model1.add(Dense(50))
    #model1.add(Dropout(0.2))
    model1.add(Activation("relu"))
    
    model1.add(Dense(1))
    
    
    model1.summary()
    
    model1.compile(loss="mse",optimizer=Adam(lr= 0.001))
    
    



    
    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_1 (Cropping2D)        (None, 65, 320, 3)    0           cropping2d_input_1[0][0]         
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 66, 200, 3)    0           cropping2d_1[0][0]               
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 66, 200, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 17, 50, 16)    3088        lambda_2[0][0]                   
___________________________________________________________________________________________

In [16]:
batch_size=32
nb_epoch=20

checkpointer= ModelCheckpoint(filepath="./Checkpointer/comma-4c.{epoch:02d}-{val_loss:.2f}.hdf5",verbose=1,save_best_only=False)


In [17]:
model1.fit_generator(train_generator,
                     samples_per_epoch=len(train_samples),
                     validation_data=validation_generator,
                     nb_val_samples=len(validation_samples),
                     nb_epoch=nb_epoch,
                     callbacks=[checkpointer],
                     verbose=1)

print("Training done...")

Epoch 1/20
7596/7634 [============================>.] - ETA: 0s - loss: 0.0144

/Users/likangning/anaconda/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: saving model to ./Checkpointer/comma-4c.00-0.03.hdf5
7722/7634 [==============================] - 56s - loss: 0.0143 - val_loss: 0.0310
Epoch 2/20
7740/7634 [==============================] - 53s - loss: 0.0111 - val_loss: 0.0461
Epoch 3/20
7740/7634 [==============================] - 52s - loss: 0.0080 - val_loss: 0.0246
Epoch 4/20
7746/7634 [==============================] - 51s - loss: 0.0138 - val_loss: 0.0236
Epoch 5/20
7698/7634 [==============================] - 52s - loss: 0.0133 - val_loss: 0.0112
Epoch 6/20
7656/7634 [==============================] - 54s - loss: 0.0154 - val_loss: 0.0404
Epoch 7/20
7644/7634 [==============================] - 55s - loss: 0.0137 - val_loss: 0.0547
Epoch 8/20
7764/7634 [==============================] - 56s - loss: 0.0099 - val_loss: 0.0465
Epoch 9/20
7794/7634 [==============================] - 53s - loss: 0.0103 - val_loss: 0.0126
Epoch 10/20
7668/7634 [==============================] - 55s - loss: 0.0118 - val_loss: 0.0132
Epoc

In [18]:
model_json=model1.to_json()
with open("model.json","w") as json_file:
     json_file.write(model_json)
print("saving model.json..")


saving model.json..


In [19]:
model1.save("model.h5")
print("done")

done
